In [25]:
import pandas as pd
import json

from IPython.display import display

#### Uploading data


In [2]:
Path = "data/"

circuits = pd.read_csv(Path + "circuits.csv")

constructor_standings = pd.read_csv(Path + "constructor_standings.csv")
constructors = pd.read_csv(Path + "constructors.csv")
driver_standings = pd.read_csv(Path + "driver_standings.csv")
drivers = pd.read_csv(Path + "drivers.csv")
lap_times = pd.read_csv(Path + "lap_times.csv")
pit_stops = pd.read_csv(Path + "pit_stops.csv")
# qualifying = pd.read_csv(Path + "qualifying.csv")
races = pd.read_csv(Path + "races.csv")
results = pd.read_csv(Path + "results.csv")
# sprint_results = pd.read_csv(Path + "sprint_results.csv")
status = pd.read_csv(Path + "status.csv")


In [3]:
driver_standings[driver_standings["driverId"] == 1]


,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
8,9,19,1,14.0,1,1,1
26,27,20,1,14.0,3,3,1
47,48,21,1,20.0,2,2,1
68,69,22,1,28.0,3,3,1
...,...,...,...,...,...,...,...
33811,71740,1093,1,198.0,6,6,0
33828,71762,1094,1,216.0,5,5,0
33840,71784,1095,1,240.0,5,5,0
33862,71806,1096,1,240.0,6,6,0


In [4]:
circuits = circuits[["circuitId", "name", "country", "location", "lat", "lng", "alt"]]
constructor_standings = constructor_standings[
    ["raceId", "constructorId", "points", "position", "wins"]
]
constructors = constructors[["constructorId", "name"]]
driver_standings = driver_standings[
    ["raceId", "driverId", "points", "position", "wins"]
]

name = drivers["forename"] + " " + drivers["surname"]
drivers = drivers[["driverId", "number", "code"]]
drivers["name"] = name

races = races[["raceId", "circuitId", "year", "date", "name"]]
results = results[["raceId", "driverId", "constructorId", "position", "points"]]


### Data processing for drivers


#### driver_teammates

Generate a dictionary (JSON file) where every driver has a list of drivers they raced with (raced for the same constructor during the same year).


In [ ]:
# Merge files to get relevant information and group by (constructorId, year)
merged = pd.merge(
    results[["raceId", "driverId", "constructorId"]],
    races[["raceId", "year"]],
    on=["raceId"],
    how="inner",
)
merged = pd.merge(merged, drivers[["driverId", "name"]], on=["driverId"], how="inner")
constructor_list = (
    merged[["name", "constructorId", "year"]]
    .drop_duplicates()
    .groupby(["constructorId", "year"])["name"]
    .apply(list)
    .to_dict()
)
constructor_list


In [ ]:
# Create the driver dictionary
driver_list = {}
for (constructorId, year), names in constructor_list.items():
    for name in names:
        if name not in driver_list:
            driver_list[name] = []
        teammates = []
        for n in names:
            if n != name and n not in driver_list[name]:
                teammates.append(n)
        driver_list[name].extend(teammates)
driver_list

In [ ]:
# Format
driver_teammates = []

for name, teammates in driver_list.items():
    driver = {"id": name, "data": teammates}
    driver_teammates.append(driver)

print(driver_teammates)

file_path = "website/src/data/driver_teammates.json"
with open(file_path, "w") as file:
    json.dump(driver_teammates, file)


#### data_points

Generate a dictionary (JSON file) where every driver has their cumulated points during a year, for each year


In [ ]:
# Merge files to get relevant information
merged = pd.merge(
    driver_standings[["raceId", "driverId", "points"]],
    races[["raceId", "year", "date", "name"]],
    on=["raceId"],
    how="inner",
).rename(columns={"name": "race_name"})
merged = pd.merge(merged, drivers[["driverId", "name"]], on=["driverId"], how="inner")[
    ["year", "name", "date", "race_name", "points"]
].set_index("year")

# merged[merged['year'] == 2021]
merged[merged["race_name"] == "Qatar Grand Prix"].loc[2021]


In [ ]:
# Format
formatted_data = []
for year, group in merged.groupby(["year"]):
    year_data = []
    for name, name_data in group.groupby(["name"]):
        data = [
            {"x": row["date"], "y": int(row["points"])}
            for _, row in name_data.iterrows()
        ]
        year_data.append({"id": name, "data": data})
    formatted_data.append({"year": int(year), "year_data": year_data})


print(formatted_data)

file_path = "website/src/data/data_points.json"
with open(file_path, "w") as file:
    json.dump(formatted_data, file)


#### data_rank

Generate a dictionary (JSON file) where every driver has their rank for each year


In [ ]:
# Merge files to get relevant information
# driver_standings or results
merged = pd.merge(
    driver_standings[["raceId", "driverId", "position"]],
    races[["raceId", "year"]],
    on=["raceId"],
    how="inner",
)
merged = pd.merge(merged, drivers[["driverId", "name"]], on=["driverId"], how="inner")[
    ["name", "year", "position"]
]

# Group by (name, year) to get the position at the end of the year
merged = merged.groupby(["name", "year"]).agg({"position": "last"}).reset_index()

merged = merged[["name", "year", "position"]].set_index("name")

merged.loc["Fernando Alonso"]


In [ ]:
# Format
formatted_data = []
for name, group in merged.groupby(["name"]):
    data = [
        {"x": int(row["year"]), "y": int(row["position"])}
        for _, row in group.iterrows()
    ]
    formatted_data.append({"id": name, "data": data})

formatted_data.sort(key=lambda x: len(x["data"]), reverse=True)
print(formatted_data[1])

file_path = "website/src/data/data_rank.json"
with open(file_path, "w") as file:
    json.dump(formatted_data, file)


### Process lap times for all drivers over all races


For each driver, and each circuit visualize their lap times over the seasons.

To vizualise it, we use svgs for some of the circuits.

For a chosen driver and a chosen circuit, output lap times for each year. Also join the data with the driver's team for each year.


In [27]:
file_prefix = "website/src/data/race/"

In [92]:
# list of locations with available tracks svgs
locations_svg = [
    "Abu Dhabi",
    "Melbourne",
    "Spielberg",
    "Baku",
    "Sakhir",
    "Spa",
    "São Paulo",
    "Montreal",
    "Shanghai",
    "Le Castellet",
    "Silverstone",
    "Budapest",
    "Monza",
    "Suzuka",
    "Mexico City",
    "Monte-Carlo",
    "Zandvoort",
    "Sochi",
    "Marina Bay",
    "Montmeló",
    "Austin",
]
circuits_svg = circuits[circuits.location.isin(locations_svg)]

print(f"Number of circuits: {len(circuits_svg)}")
display(circuits_svg.head())

circuits_svg = circuits_svg[["circuitId", "name", "country", "location"]]

# add column with svg_url_suffix
circuits_svg["svg_url_suffix"] = circuits_svg["country"].apply(
    lambda x: str(x).lower().replace("uae", "abudhabi").replace("uk", "greatbritain")
)

# write to json
circuits_svg.to_json(file_prefix + "circuits.json", orient="records")

Number of circuits: 21


,circuitId,name,country,location,lat,lng,alt
0,1,Albert Park Grand Prix Circuit,Australia,Melbourne,-37.8497,144.96800,10
2,3,Bahrain International Circuit,Bahrain,Sakhir,26.0325,50.51060,7
3,4,Circuit de Barcelona-Catalunya,Spain,Montmeló,41.5700,2.26111,109
5,6,Circuit de Monaco,Monaco,Monte-Carlo,43.7347,7.42056,7
6,7,Circuit Gilles Villeneuve,Canada,Montreal,45.5000,-73.52280,13


In [115]:
# find raceIds where the circuitId is in the circuits_svg dataframe
races_with_circuits = pd.merge(
    races[["raceId", "circuitId"]][races["year"] >= 2010],
    circuits_svg[["circuitId"]],
    on=["circuitId"],
    how="inner",
)[["raceId"]]

print(f"Number of rows: {races_with_circuits.shape[0]}")
print(f"All columns: {races_with_circuits.columns.tolist()}")
races_with_circuits.head()

Number of rows: 236
All columns: ['raceId']


,raceId
0,337
1,863
2,883
3,902
4,929


In [116]:
filtered_races = pd.merge(races, races_with_circuits, on=["raceId"], how="inner")

print(f"Number of rows: {filtered_races.shape[0]}")
display(filtered_races.head())

# write to json
(
    filtered_races.groupby("circuitId")
    .apply(lambda x: x.to_dict(orient="records"))
    .to_json(
        file_prefix + "races.json",
        orient="index",
    )
)

Number of rows: 236


,raceId,circuitId,year,date,name
0,337,3,2010,2010-03-14,Bahrain Grand Prix
1,338,1,2010,2010-03-28,Australian Grand Prix
2,340,17,2010,2010-04-18,Chinese Grand Prix
3,341,4,2010,2010-05-09,Spanish Grand Prix
4,342,6,2010,2010-05-16,Monaco Grand Prix


In [117]:
filtered_results = pd.merge(results, races_with_circuits, on=["raceId"], how="inner")

print(f"Number of rows: {filtered_results.shape[0]}")
display(filtered_results.head())

Number of rows: 4617


,raceId,driverId,constructorId,position,points
0,337,4,6,1,25.0
1,337,13,6,2,18.0
2,337,1,1,3,15.0
3,337,20,9,4,12.0
4,337,3,131,5,10.0


In [118]:
# using the filtered results, get all raceIds (and corresponding name and circuitId from the filtered_races df) for each driverId
filtered_races_with_results = pd.merge(
    filtered_results,
    filtered_races[["raceId", "year", "circuitId"]],
    on=["raceId"],
    how="inner",
)

data_per_circuits_per_driver = filtered_races_with_results.groupby(
    ["driverId", "circuitId"]
).apply(lambda x: x.to_dict(orient="records"))

# write to json
data_per_circuits_per_driver.to_json(file_prefix + "results.json", orient="index")

data_per_circuits_per_driver.head()

driverId  circuitId
1         1            [{'raceId': 338, 'driverId': 1, 'constructorId...
          3            [{'raceId': 337, 'driverId': 1, 'constructorId...
          4            [{'raceId': 341, 'driverId': 1, 'constructorId...
          6            [{'raceId': 342, 'driverId': 1, 'constructorId...
          7            [{'raceId': 344, 'driverId': 1, 'constructorId...
dtype: object

In [119]:
filtered_drivers = pd.merge(
    drivers,
    filtered_results[["driverId"]].drop_duplicates(),
    on=["driverId"],
    how="inner",
)

filtered_drivers["number"] = pd.to_numeric(filtered_drivers["number"], errors="coerce")
filtered_drivers = filtered_drivers.dropna(subset=["number"])

print(f"Number of rows: {filtered_drivers.shape[0]}")
display(filtered_drivers.head())

# write to json
(
    filtered_drivers.to_json(
        file_prefix + "drivers.json",
        orient="records",
    )
)

Number of rows: 52


,driverId,number,code,name
0,1,44.0,HAM,Lewis Hamilton
2,3,6.0,ROS,Nico Rosberg
3,4,14.0,ALO,Fernando Alonso
5,8,7.0,RAI,Kimi Räikkönen
6,9,88.0,KUB,Robert Kubica


In [120]:
filtered_constructors = pd.merge(
    constructors,
    filtered_results[["constructorId"]].drop_duplicates(),
    on=["constructorId"],
    how="inner",
).set_index("constructorId")

print(f"Number of rows: {filtered_constructors.shape[0]}")
display(filtered_constructors.head())

# write to json
(
    filtered_constructors.to_json(
        file_prefix + "constructors.json",
        orient="index",
    )
)

Number of rows: 22


,name
constructorId,
1,McLaren
3,Williams
4,Renault
5,Toro Rosso
6,Ferrari


In [122]:
filtered_lap_times = pd.merge(
    lap_times,
    filtered_results[["raceId", "driverId"]].drop_duplicates(),
    on=["raceId", "driverId"],
    how="inner",
).drop(columns=["time"])

print(f"Number of rows: {filtered_lap_times.shape[0]}")
display(filtered_lap_times.head())

# write to json
(
    filtered_lap_times.groupby(["driverId", "raceId"])
    .apply(
        lambda x: x[["lap", "milliseconds"]]
        .set_index("lap")
        .sort_index()
        .to_dict(orient="list")
    )
    .to_json(
        file_prefix + "lap_times.json",
        orient="index",
    )
)

Number of rows: 244398


,raceId,driverId,lap,position,milliseconds
0,841,20,1,1,98109
1,841,20,2,1,93006
2,841,20,3,1,92713
3,841,20,4,1,92803
4,841,20,5,1,92342


---


For a given driver, how the driver's performance evolved over time, which teams they raced for, how many races did they win, compare them to other drivers, etc.

By selecting a driver -> output:

- plot for wins in time (+total)
- plot for points in time
- constructor they raced for in time
  Options:
- possibility to select up to x drivers to compare them
- short description of each driver ?


In [ ]:
merged = pd.merge(results, races[["raceId", "year"]], on=["raceId"], how="inner")
merged = pd.merge(
    merged,
    driver_standings[["raceId", "driverId", "wins"]],
    on=["raceId", "driverId"],
    how="inner",
)

merged
# merged.to_csv("processed_data/driver_results.csv")
# merged.to_json("processed_data/driver_results.json")


For each year, the schedule of the races, their locations on the world map, which teams participated, etc.

To determine.
An idea: By selecting a year -> output:

- list of races
- world map with locations
- list of constructors
  Options:
- show the movement


In [ ]:
merged = pd.merge(
    races, results[["raceId", "driverId", "constructorId"]], on=["raceId"], how="inner"
)
merged = merged[
    ["raceId", "circuitId", "driverId", "constructorId", "year", "date", "name"]
]

merged
# merged.to_csv("processed_data/schedule.csv")
# merged.to_json("processed_data/schedule.json")
